# 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


# DeepFM

In [ ]:
!pip install -U deepctr-torch

In [5]:
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

In [6]:
# df = pd.read_csv('data/total_data.csv')
# df.head(2)

In [7]:
vec_df = pd.read_csv('data/vec512.csv', low_memory=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
from sklearn.metrics import roc_auc_score

grouped_label = vec_df.groupby('label').size()
average_ctr = float(grouped_label[1]/grouped_label.sum())
average_ctr

def get_rig(train_y, test_y, pred):
    avg_ctr = average_ctr
    prior = log_loss(train_y, [avg_ctr]*len(train_y))

    classifier = log_loss(test_y, pred)

    rig = (prior - classifier) / prior
    return rig


In [13]:
sparse_features = [
    'viewer_gender',  
    'content_used',
    'content_cat_1',
    'content_cat_2',
    'content_cat_3',
    "content_status"]

dense_features = [
    'bid_price', 
    'content_price',
    'content_emergency_count',
    'content_comment_count',     
    'content_views',       
    'content_likes',    
    'adv_follower_count',
    'adv_grade',       
    'adv_item_count',         
    'adv_views', # adv_reivew drop
    'adv_review_count',
    'adv_comment_count',
    'adv_pay_count',
    'adv_parcel_post_count', 
    'adv_transfer_count', 
    'adv_chat_count',
    'viewer_age',
    'viewer_following_count',
    'viewer_pay_count',
    #"viewer_trans_pay_count", # viewer_trans drop
    'viewer_transfer_count',
    'viewer_chat_count']

vec_features = [str(i) for i in range(512)]

In [14]:
vec_df[sparse_features] = vec_df[sparse_features].fillna('-1', )
vec_df[dense_features] = vec_df[dense_features].fillna(0, )
target = ['label']

In [15]:
vec_df.head(2)

,content_id,bid_price,imp_id,imp_time,label,user_id,view_time,adv_id,content_name,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,content_delivery_fee,content_place,content_status,adv_follower_count,adv_grade,adv_item_count,adv_views,adv_review_count,adv_comment_count,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count,content_img_url,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,137078818,50,99d1612dded122458e99,16:48:41.780,1,1728109,16:56:46.013,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,64,600700001,1,4,25583,817,0,NaN,NaN,삭제됨,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,https://media.bunjang.co.kr/product/137078818_...,...,0.033936,3.04929,1.684554,0.484026,2.723213,1.283192,0.258292,0.886391,1.344672,0.676913,6.133104,0.226652,2.635666,0.298511,0.509246,0.038719,0.058445,0.396445,0.016248,0.539511,0.061262,0.749012,0.091903,0.048339,0.301149,0.414436,0.0,0.470932,0.144566,0.037908,0.450609,0.379989,0.09685,0.451598,0.06229,0.021293,0.626392,0.310855,0.15655,0.010067
1,137078818,50,971a612de1a50ebc01ce,17:00:45.805,0,1728109,NaN,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,64,600700001,1,4,25583,817,0,NaN,NaN,삭제됨,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,https://media.bunjang.co.kr/product/137078818_...,...,0.033936,3.04929,1.684554,0.484026,2.723213,1.283192,0.258292,0.886391,1.344672,0.676913,6.133104,0.226652,2.635666,0.298511,0.509246,0.038719,0.058445,0.396445,0.016248,0.539511,0.061262,0.749012,0.091903,0.048339,0.301149,0.414436,0.0,0.470932,0.144566,0.037908,0.450609,0.379989,0.09685,0.451598,0.06229,0.021293,0.626392,0.310855,0.15655,0.010067


In [16]:
# sparse 데이터는 Label Encoding
for feat in sparse_features:
    lbe = LabelEncoder()
    vec_df[feat] = lbe.fit_transform(vec_df[feat])
    
# dense 데이터는 Min-Max Scaling
# scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler()

vec_df[dense_features] = scaler.fit_transform(vec_df[dense_features])
vec_df[vec_features] = scaler.fit_transform(vec_df[vec_features])

In [18]:
# 3.generate input data for model

train, test = train_test_split(vec_df, test_size=0.2, random_state=47)#, stratify=df['label'])

In [23]:
# 2.count #unique features for each sparse field,and record dense feature field name

# 각 feature마다 값의 종류가 몇개인지, feature 이름이 무엇인지 등 저장
fixlen_feature_columns = [SparseFeat(feat, vec_df[feat].nunique())
                            for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                            for feat in dense_features]
vec_columns = [DenseFeat(feat, 1, ) for feat in vec_features]

# dnn feature에 일단 모든 feature 정보 넣기
dnn_feature_columns = fixlen_feature_columns # + vec_columns

# lenear feature에도 일단 모든 feature 정보 넣기
linear_feature_columns = fixlen_feature_columns # + vec_columns

# feature 이름들만 따로 저장
feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

In [24]:
# train/test 데이터의 각 feature에 대응하는 컬럼을 딕셔너리로 저장
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

- Standard Scaling, 이미지 벡터 제외

In [25]:
# 4.Define Model,train,predict and evaluate
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

# optimizer, loss 설정
model.compile("adagrad", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )
# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=1024, epochs=20, verbose=2, validation_split=0.0)

# 예측
pred_ans = model.predict(test_model_input, 1024)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
cuda:0
Train on 588555 samples, validate on 0 samples, 575 steps per epoch
Epoch 1/20
8s - loss:  0.1388 - binary_crossentropy:  0.1388 - auc:  0.6745
Epoch 2/20
8s - loss:  0.1350 - binary_crossentropy:  0.1350 - auc:  0.6879
Epoch 3/20
8s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6896
Epoch 4/20
8s - loss:  0.1346 - binary_crossentropy:  0.1346 - auc:  0.6899
Epoch 5/20
8s - loss:  0.1346 - binary_crossentropy:  0.1346 - auc:  0.6918
Epoch 6/20
8s - loss:  0.1345 - binary_crossentropy:  0.1344 - auc:  0.6924
Epoch 7/20
8s - loss:  0.1344 - binary_crossentropy:  0.1344 - auc:  0.6931
Epoch 8/20
8s - loss:  0.1343 - binary_crossentropy:  0.1343 - auc:  0.6942
Epoch 9/20
8s - loss:  0.1342 - binary_crossentropy:  0.1342 - auc:  0.6941
Epoch 10/20
8s - loss:  0.1342 - binary_crossentropy:  0.1342 - auc:  0.6956
Epoch 11/20
8s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.6955
Epoch 12/20
8s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.6

- Standard Scaling, 이미지 벡터 전부 사용

In [22]:
# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=1024, epochs=20, verbose=2, validation_split=0.0)

# 예측
pred_ans = model.predict(test_model_input, 1024)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda:0
Train on 588555 samples, validate on 0 samples, 575 steps per epoch
Epoch 1/20
11s - loss:  0.1303 - binary_crossentropy:  0.1303 - auc:  0.7360
Epoch 2/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7356
Epoch 3/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7365
Epoch 4/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7366
Epoch 5/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7366
Epoch 6/20
11s - loss:  0.1301 - binary_crossentropy:  0.1301 - auc:  0.7376
Epoch 7/20
11s - loss:  0.1301 - binary_crossentropy:  0.1301 - auc:  0.7374
Epoch 8/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7381
Epoch 9/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7387
Epoch 10/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7386
Epoch 11/20
11s - loss:  0.1299 - binary_crossentropy:  0.1299 - auc:  0.7391
Epoch 12/20
11s - loss:  0.1299 - binary_crossentropy:  0.1299 - auc:  0.739